<table align='right'><tr>
<td style="padding:10px"><img src="resources/img/logos/EC_POS.png" style="max-height:50px;width:auto;"/></td>
<td style="padding:10px"><img src="resources/img/logos/ESA_logo_2020_Deep.png" style="max-height:40px;width:auto;"/></td>
<td style="padding:10px"><img src="resources/img/logos/Copernicus_blue.png" style="max-height:60px;width:auto;"/></td>
<td style="padding:10px"><img src="resources/img/logos/AIRBUS_Blue.png" style="max-height:30px;width:auto;"/></td>
<td style="padding:10px"><img src="resources/img/logos/CS-GROUP.png" style="max-height:50px;width:auto;"/></td>
</tr></table>

<a href="./ESA_checkpoint_v0.1_01_initialisation.ipynb" target="_blank"><< Part 1: initialisation</a>
<br>
<a href="./ESA_checkpoint_v0.1_03_stac_catalog.ipynb" target="_blank">Part 3: use the STAC catalog >></a>

<font color="#138D75">**Copernicus Reference System Python**</font> <br>
**Copyright:** Copyright 2024 ESA <br>
**License:** Apache License, Version 2.0 <br>
**Authors:** Airbus, CS Group

<div class="alert alert-block alert-success">
<h3>Copernicus Reference System Python tutorial for the ESA checkpoint 0.1</h3></div>

<div class="alert alert-block alert-warning">

<h4>Part 2: manual calls to the RS-Server services</h4>

Prerequisites:
* <a href="./ESA_checkpoint_v0.1_01_initialisation.ipynb" target="_blank">Part 1: initialisation</a>

</div>
<hr>

# Introduction

## Links

* GitHub: https://github.com/RS-PYTHON
* Documentation: https://home.rs-python.eu/rs-documentation/

## Data used

In this notebook, we use simulated Auxip and Cadip data.

## Learning outcomes

At the end of this notebook you will know how to:
* Use the RS-Client Python library.
* Use it to:
    * Search individual Auxip files and Cadip chunk files from the Auxip and Cadip simulators.
    * Download them into a temporary S3 bucket (=first staging step).
    * Search Cadip session information.

<div class="alert alert-info" role="alert">

## Contents

</div>
    
1. [Check your installation](#Check-your-installation) 
1. [RsClient initialisation](#RsClient-initialisation)
1. [Call services manually](#Call-services-manually)
    1. [Search files from Auxip and Cadip stations](#Search-files-from-Auxip-and-Cadip-stations)
    1. [Stage Auxip and Cadip files](#Stage-Auxip-and-Cadip-files)
    1. [Search Cadip sessions](#Search-Cadip-sessions)
1. [Exercises](#Exercises)

<hr>

<div class="alert alert-info" role="alert">

## Check your installation

In this section, we will check that your Jupyter Notebook environment is correctly set.

[Back to top](#Contents)

</div>

### `rs-client-libraries` installation

The `rs-client-libraries` Python library is the preferred way to access the RS-Server services from your environment. It is automatically installed in this notebook.

**Note**: don't worry about these OpenTelemetry messages for now, they will be fixed in a later version:
```
Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented
Transient error StatusCode.UNAVAILABLE encountered while exporting metrics to ..., retrying in ...s
Failed to export metrics to ..., error code: StatusCode.UNIMPLEMENTED
```

In [ ]:
import rs_client
import rs_common
import rs_workflows

# Set logger level to info
import logging
rs_common.logging.Logging.level = logging.INFO

### Environment

In [ ]:
import os

# In local mode, all your services are running locally.
# In hybrid or cluster mode, we use the services deployed on the RS-Server website.
# This configuration is set in an environment variable.
local_mode = (os.getenv("RSPY_LOCAL_MODE") == "1")

# In local mode, the service URLs are hardcoded in the docker-compose file
if local_mode:
    rs_server_href = None # not used
    RSPY_HOST_AUXIP = "http://localhost:8001/docs"
    RSPY_HOST_CADIP = "http://localhost:8002/docs"
    RSPY_HOST_CATALOG = "http://localhost:8003/api.html"

# In hybrid or cluster mode, they are set in an environment variables
else:
    rs_server_href = os.environ["RSPY_WEBSITE"]

### API key

In [ ]:
apikey = os.getenv("RSPY_APIKEY")
if (not local_mode) and (not apikey):
    import getpass
    apikey = getpass.getpass(f"Enter your API key:")
    os.environ["RSPY_APIKEY"] = apikey

<div class="alert alert-info" role="alert">

## RsClient initialisation

Initialise Python RsClient class instances to access the RS-Server services.

[Back to top](#Contents)

</div>

In [ ]:
import json
from rs_client.rs_client import RsClient
from rs_common.config import ECadipStation

# Init a generic RS-Client instance. Pass the:
#   - RS-Server website URL
#   - API key. If not set, we try to read it from the RSPY_APIKEY environment variable.
#   - ID of the owner of the STAC catalog collections.
#     By default, this is the user login from the keycloak account, associated to the API key.
#     Or, in local mode, this is the local system username.
#     Else, your API Key must give you the rights to read/write on this catalog owner (see next cell).
#   - Logger (optional, a default one can be used)
generic_client = RsClient(rs_server_href, rs_server_api_key=None, owner_id=None, logger=None)
print(f"STAC catalog owner: {generic_client.owner_id!r}")

# From this generic instance, get an Auxip client instance
auxip_client = generic_client.get_auxip_client()

# Or get a Cadip client instance. Pass the cadip station.
cadip_station = ECadipStation.CADIP # you can also have: INS, MPS, MTI, NSG, SGS
cadip_client = generic_client.get_cadip_client(cadip_station)

# Or get a Stac client to access the catalog
stac_client = generic_client.get_stac_client()

print("\nValidate that our catalog is valid to the STAC format...")
stac_client.validate_all()

print("\nDisplay the Stac catalog as a treeview in notebook:")
display(stac_client)

print("\nOr just display all its contents at once:")
print(json.dumps(stac_client.to_dict(), indent=2))

In [ ]:
# In hybrid or cluster mode, show information from the keycloak account, associated to the api key
if not local_mode:

    # = keycloak account user login
    print(f"API key user login: {generic_client.apikey_user_login!r}")

    # Print the IAM (Identity and Access Management) roles
    # For this tutorial, you must have: 
    #   - read/download access for Adgs (=Auxip) = "rs_adgs_<read|download>"
    #   - read/download access to the Cadip station you passed on the above cell = "rs_cadip_<station>_<read|download>"
    #   - (optional) read/write/download access to STAC catalog collections from other owners = "rs_catalog_<owner_id>:<collection|*>_<read|write|download>"
    #     (you always have all access to your own collections with owner_id=apikey_user_login as printed above)
    iam_roles = "\n".join (sorted (generic_client.apikey_iam_roles))
    print(f"\nAPI key IAM roles: \n{iam_roles}")

<div class="alert alert-info" role="alert">

## Call services manually

In this section, we will see how to call manually these services: 

  * Search Auxip and Cadip reception stations for new files
  * Stage these files (in a temporary s3 bucket, not yet in the STAC catalog) and check the staging status
  * Search Cadip sessions

[Back to top](#Contents)

</div>

In [ ]:
# Do some initialisation
from datetime import datetime
import json
from time import sleep

from rs_common.config import EDownloadStatus, EPlatform

# Define a search interval
start_date = datetime(2010, 1, 1, 12, 0, 0)
stop_date = datetime(2024, 1, 1, 12, 0, 0)

# We use this bucket name that is deployed on the cluster. 
# RS-Server has read/write access to this bucket, but as an end-user, you won't manipulate it directly.
RSPY_TEMP_BUCKET = os.environ["RSPY_TEMP_BUCKET"]

<div class="alert alert-info" role="alert">

### Search files from Auxip and Cadip stations

[Back to top](#Contents)

</div>

![Search Auxip and Cadip stations](resources/img/v0.1/search_stations.drawio.png)

In [ ]:
# Do this using the Auxip client (to find Auxip files) 
# then the Cadip client (to find Cadip chunk files)
for client in [auxip_client, cadip_client]:

    # Call the service to search the reception stations for new files in the date interval.
    files = client.search_stations(start_date, stop_date)
    
    file_count = len(files)
    assert file_count, f"We should have at least one {client.station_name} file"
    print (f"Found {file_count} {client.station_name} files\n")

    # Print the first file metadata. It is in the STAC format.
    print(f"First {client.station_name} file:\n{json.dumps(files[0], indent=2)}\n")

    # By default, the files are returned sorted by the most recent first.
    # NOTE: the "created" field corresponds to the publication date.
    ids="\n".join([f"{f['properties']['created']} - {f['id']}" for f in files[:10]])
    print(f"Most recent {client.station_name} IDs and datetimes:\n{ids}\n")

In [ ]:
# We can sort by +/- any property, e.g. by creation date ascending = the oldest first
for client in [auxip_client, cadip_client]:    
    files = client.search_stations(start_date, stop_date, sortby="+created")
    ids="\n".join([f"{f['properties']['created']} - {f['id']}" for f in files[:10]])
    print(f"Oldest {client.station_name} IDs and datetimes:\n{ids}\n")

<div class="alert alert-info" role="alert">

### Stage Auxip and Cadip files

When RS-Server stages a file, it means to:
1. Copy (=download) it from the reception station into the temporary S3 bucket.
1. Publish its metadata into the STAC catalog and move it from the temporary into the final S3 bucket.

Here we will only perform the first step. The second step will be done in the next tutorial.

[Back to top](#Contents)

</div>

![Staging step #1](resources/img/v0.1/staging_step1.drawio.png)

In [ ]:
temp_s3_files = []
for client in [auxip_client, cadip_client]:

    # When searching stations, we can also limit the number of returned results.
    # For this example, let's keep only one file.
    files = client.search_stations(start_date, stop_date, limit=1)
    assert len(files) == 1

    # We stage by filename = the file ID
    first_filename = files[0]["id"]

    # We must give a temporary S3 bucket path where to copy the file from the station.
    # Use our API key username so avoid conflicts with other users.
    # NOTE: in future versions, this S3 path will be automatically calculated by RS-Server.
    s3_path = f"s3://{RSPY_TEMP_BUCKET}/{client.apikey_user_login}/{client.station_name}"
    temp_s3_files.append (f"{s3_path}/{first_filename}") # save it for later

    # We can also download the file locally to the server, but this is useful only in local mode
    local_path = None

    # Call the staging service
    client.staging(first_filename, s3_path=s3_path, tmp_download_path=local_path)

    # Then we can check when the staging has finished by calling the check status service
    while True:
        status = client.staging_status(first_filename)
        print (f"Staging status for {first_filename!r}: {status.value}")
        if status in [EDownloadStatus.DONE, EDownloadStatus.FAILED]:
            print("\n")
            break
        sleep(1)        
    assert status == EDownloadStatus.DONE, "Staging has failed"

    # WARNING: the file is copied into the temporary S3 bucket but is not yet published into the catalog

# Save these values for the next notebooks
%store temp_s3_files

<div class="alert alert-info" role="alert">

### Search Cadip sessions

All Cadip chunk files are attached to a single Cadip session.

We can search Cadip sessions by parameters, or find information about a specific session ID.

[Back to top](#Contents)

</div>

![Search Cadip sessions](resources/img/v0.1/search_cadip_sessions.drawio.png)

In [ ]:
# Search cadip sessions by date interval and platforms
platforms = [EPlatform.S1A, EPlatform.S2B]
sessions = cadip_client.search_sessions(start_date=start_date, stop_date=stop_date, platforms=platforms)

session_count = len(sessions)
assert session_count, "We should have at least one Cadip session"
print (f"Found {session_count} Cadip sessions")

In [ ]:
# Print the first cadip session metadata. It is in the STAC format.
print(f"First Cadip session:\n{json.dumps(sessions[0], indent=2)}\n")

In [ ]:
# Print all the Cadip session IDs
ids=[s["id"] for s in sessions]
print_ids="\n".join(ids)
print(f"Cadip sessions IDs:\n{print_ids}")

In [ ]:
# We can also search Cadip sessions by specific session IDs,
# e.g. get information for the cadip sessions #2 and #3
search_ids=ids[1:3]
search_sessions = cadip_client.search_sessions(session_ids=search_ids)
print(f"Cadip sessions information:\n{json.dumps(search_sessions, indent=2)}\n")

<div class="alert alert-danger" role="alert">

## Exercises

</div>

Run again the previous cells but this time: 
1. Check if your **API key** allows you to access **other Cadip stations**. Then:
    1. Use a RsClient instance with one of these **other stations**.
    1. Check if it allows you to read and download **Cadip files**.
1. Search Auxip and Cadip files using a different time interval.
1. Print one different Cadip session information.

**NOTE**: you can also use the website OpenAPI Swagger UI to call RS-Server (see cell below).

[Back to top](#Contents)

<hr>

In [ ]:
if local_mode:
    print(f"""OpenAPI Swagger UI for:
  - Auxip: {RSPY_HOST_AUXIP}
  - Cadip: {RSPY_HOST_CADIP}
  - STAC catalog: {RSPY_HOST_CATALOG}""")
else:
    print(f"OpenAPI Swagger UI: {generic_client.rs_server_href}")

<a href="./ESA_checkpoint_v0.1_01_initialisation.ipynb" target="_blank"><< Part 1: initialisation</a>
<br>
<a href="./ESA_checkpoint_v0.1_03_stac_catalog.ipynb" target="_blank">Part 3: use the STAC catalog >></a>

<hr>
<a href="https://github.com/RS-PYTHON" target="_blank">View on GitHub</a>